In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys,os
print(os.getcwd())

/home/cayan/Documentos/Trabalhos/pd_imbalanced/notebooks


In [3]:
# Classifier Libraries
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate, RandomizedSearchCV, GridSearchCV, KFold, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, classification_report

#import collections

# Other Libraries
from imblearn.pipeline import make_pipeline as imbalanced_make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss, RandomUnderSampler
from imblearn.ensemble import BalancedBaggingClassifier, BalancedRandomForestClassifier
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import Pipeline as imb_pipelineb
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline as imb_pipeline
from imblearn.under_sampling import TomekLinks

In [4]:
# GroupTimeSeriesSplit
from mlxtend.evaluate.time_series import (
    GroupTimeSeriesSplit,
    plot_splits,
    print_cv_info,
    print_split_info
)

In [5]:
from src.data.load_data import read_params, read_data

In [7]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)
pd.options.display.float_format = '{:.2f}'.format

In [8]:
config = read_params(config_path="../params.yaml")

safra_config = config["proc_data_config"]["safra_config"]
shuffle_config = config["proc_data_config"]["shuffle_config"]

In [9]:
x_safra_train = read_data('../'  + safra_config["x_train_csv"])
y_safra_train = read_data('../'  + safra_config["y_train_csv"])

x_shuffle_train = read_data('../'  + shuffle_config["x_train_csv"])
y_shuffle_train = read_data('../'  + shuffle_config["y_train_csv"])

In [8]:
from src.models.parameters import (
    # safra
    safra_best_parameters_bbagg,
    safra_best_parameters_smote,
    safra_best_parameters_tomek,
    # shuffle
    shuffle_best_parameters_bbagg,
    shuffle_best_parameters_smote,
    shuffle_best_parameters_tomek
)

In [14]:
safra_split = {
        'safra_bbagg': safra_best_parameters_bbagg,
        'safra_smote': safra_best_parameters_smote,
        'safra_tomek': safra_best_parameters_tomek
}

shuffle_split = {
        'shuffle_bbagg': shuffle_best_parameters_bbagg,
        'shuffle_smote': shuffle_best_parameters_smote,
        'shuffle_tomek': shuffle_best_parameters_tomek
}

In [16]:
juntos = [safra_split, shuffle_split]

In [32]:
step_cen = {
        "pass": ("pass", "passthrough", [0]),
        "onehot_instrucao": ("cat", OneHotEncoder(), [1]),
        "std_all": ("num", StandardScaler(), [2, 3, 4, 5, 6]),
        "std_not_idade": ("num", StandardScaler(), [3, 4, 5, 6])
    }

preproc_i = ColumnTransformer(
            transformers=[step_cen["pass"], step_cen["onehot_instrucao"], step_cen["std_all"]],
            remainder="drop"
            )

In [33]:
test_pipeline = imb_pipeline([
    ('pre_processor', preproc_i),
    ('smote', SMOTE(n_jobs=-1)),
    ('svc', SVC(kernel='linear', C=0.9, probability=True))
])

In [34]:
test_pipeline.fit(x_safra_train, y_safra_train)

/home/cayan/.pyenv/versions/3.10.0/envs/pyenv_pd_imb/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
/home/cayan/.pyenv/versions/3.10.0/envs/pyenv_pd_imb/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('pre_processor',
                 ColumnTransformer(transformers=[('pass', 'passthrough', [0]),
                                                 ('cat', OneHotEncoder(), [1]),
                                                 ('num', StandardScaler(),
                                                  [2, 3, 4, 5, 6])])),
                ('smote', SMOTE(n_jobs=-1)),
                ('svc', SVC(C=0.9, kernel='linear', probability=True))])

In [35]:
from sklearn.svm import LinearSVC
#clf = LinearSVC(random_state=0, tol=1e-5)
#clf.fit(X_train, y_train.ravel()) 

In [62]:
test_pipeline_linearsvc = imb_pipeline([
    ('pre_processor', preproc_i),
    ('smote', SMOTE(n_jobs=-1)),
    ('svc', LinearSVC(random_state=0, C=1000, tol=1e-5))
])

In [ ]:
y_safra_train.default_ate_12m

In [63]:
test_pipeline_linearsvc.fit(x_safra_train, y_safra_train.default_ate_12m)

/home/cayan/.pyenv/versions/3.10.0/envs/pyenv_pd_imb/lib/python3.10/site-packages/imblearn/over_sampling/_smote/base.py:336: FutureWarning: The parameter `n_jobs` has been deprecated in 0.10 and will be removed in 0.12. You can pass an nearest neighbors estimator where `n_jobs` is already set instead.
  warnings.warn(
/home/cayan/.pyenv/versions/3.10.0/envs/pyenv_pd_imb/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('pre_processor',
                 ColumnTransformer(transformers=[('pass', 'passthrough', [0]),
                                                 ('cat', OneHotEncoder(), [1]),
                                                 ('num', StandardScaler(),
                                                  [2, 3, 4, 5, 6])])),
                ('smote', SMOTE(n_jobs=-1)),
                ('svc', LinearSVC(C=1000, random_state=0, tol=1e-05))])

Resultados dos modelo

In [ ]:
x_safra_test = read_data('../'  + safra_config["x_test_csv"])
y_safra_test = read_data('../'  + safra_config["y_test_csv"])

x_shuffle_test = read_data('../'  + shuffle_config["x_test_csv"])
y_shuffle_test = read_data('../'  + shuffle_config["y_test_csv"])

In [7]:
import pickle

In [27]:
pkl_file = '/home/cayan/Documentos/Trabalhos/pd_imbalanced/models/safra/safra_smote_cenario_1_modelo_4.pkl'

with open(pkl_file, 'rb') as handle:
     mod = pickle.load(handle)

In [28]:
mod

Pipeline(steps=[('pre_processor',
                 ColumnTransformer(transformers=[('pass', 'passthrough', [0]),
                                                 ('cat', OneHotEncoder(), [1]),
                                                 ('num', StandardScaler(),
                                                  [2, 3, 4, 5, 6])])),
                ('smote', SMOTE(n_jobs=-1)),
                ('clf',
                 RandomForestClassifier(max_depth=2, min_samples_leaf=2,
                                        min_samples_split=20,
                                        n_estimators=10))])

In [29]:
preditos = mod.predict(x_safra_test)

In [64]:
preditos_test = test_pipeline.predict(x_safra_test)
preditos_test_linearsvc = test_pipeline_linearsvc.predict(x_safra_test)

In [15]:
from imblearn.metrics import sensitivity_specificity_support, sensitivity_score, classification_report_imbalanced

In [65]:
report_test = classification_report_imbalanced(
    y_safra_test.default_ate_12m, preditos_test,
    output_dict=True
    )

report_linearsvc = classification_report_imbalanced(
    y_safra_test.default_ate_12m, preditos_test_linearsvc,
    output_dict=True
    )

In [66]:
pd.DataFrame.from_dict(report_test[1], orient='index').T
pd.DataFrame.from_dict(report_linearsvc[1], orient='index').T

,pre,rec,spe,f1,geo,iba,sup
0,0.08,0.64,0.54,0.15,0.59,0.35,594.00


,pre,rec,spe,f1,geo,iba,sup
0,0.06,0.23,0.75,0.09,0.42,0.17,594.00


In [41]:
pd.DataFrame.from_dict(report_0[1], orient='index').T

,pre,rec,spe,f1,geo,iba,sup
0,0.08,0.64,0.55,0.15,0.60,0.36,594.00


In [26]:
pd.DataFrame.from_dict(report_0[1], orient='index').T

,pre,rec,spe,f1,geo,iba,sup
0,0.08,0.63,0.56,0.15,0.60,0.36,594.00
